# Python Script to create PDF from dashboard based on parameters
## Importing libraries and features

In [ ]:
pip install PyPDF2

In [22]:
import os, keyring, re, configparser, warnings, urllib, requests, shutil, json
from PyPDF2 import PdfFileMerger, PdfFileReader
from fpdf import FPDF
from PIL import Image
import math
import datetime
from getpass import getpass
from pathlib import Path
warnings.filterwarnings('ignore')
config = configparser.ConfigParser()
connection = configparser.ConfigParser()
config.read(r".\tableau_server_pdf_generator_config.cfg")
connection.read(r".\tableau_server_pdf_generator_connection.cfg")
illegal_chars = config["logging_details"]["illegalchars"].split(',')

## Open Log file

In [2]:
log_file_loc = r"{}\{}".format(str(Path.home()), config["logging_details"]["logfilename"])
log_file = open(log_file_loc, "a+")

## Functions
### Fn to check errors

In [3]:
def check_error(request, task):
    if task == "sign_in":
        if request.status_code == 200:
            print("\t\tUser signed in successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: User not found!!")
            return 0
        elif request.status_code == 401:
            print("\t\tERROR: Login error!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "sign_out":
        if request.status_code == 204:
            print("\t\tUser signed out successfully!")
            return 1
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "create_users":
        if request.status_code == 201:
            print("\t\tUser created successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: Site not found!!")
            return 0
        elif request.status_code == 409:
            print("\t\tERROR: User exists or license unavailable, check again!!")
            return 0
        elif request.status_code == 400:
            print("\t\tERROR: Invalid site role or bad request!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "update_users":
        if request.status_code == 200:
            print("\t\tUser information updated successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: User or Site not found!!")
            return 0
        elif request.status_code == 409:
            print("\t\tERROR: User exists or license unavailable, check again!!")
            return 0
        elif request.status_code == 400:
            print("\t\tERROR: Invalid site role, email address or bad request!!")
            return 0
        elif request.status_code == 403:
            print("\t\tERROR: Licensing update on self or guest account not allowed!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "find_group_id":
        if request.status_code == 200:
            print("\t\tGroup found!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: Site not found!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "add_user_group":
        if request.status_code == 200:
            print("\t\tUser added to group successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: Site or Group not found!!")
            return 0
        elif request.status_code == 409:
            print("\t\tERROR: Specified User already in group!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
    elif task == "query_workbooks_site":
        if request.status_code == 200:
            print("\t\tQueried workbook name successfully!")
            return 1
        elif request.status_code == 400:
            print("\t\tERROR: Pagination error!!")
            return 
        elif request.status_code == 403:
            print("\t\tERROR: Forbidden to read workbook!!")
            return 0
        elif request.status_code == 404:
            print("\t\tERROR: Site or workbook not found!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
    elif task == "query_view_image":
        if request.status_code == 200:
            print("\t\tQueried view image successfully!")
            return 1
        elif request.status_code == 403:
            print("\t\tERROR: Forbidden to view image!!")
            return 0
        elif request.status_code == 404:
            print("\t\tERROR: Site, workbook or view not found!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
    elif task == "query_view_pdf":
        if request.status_code == 200:
            print("\t\tQueried view pdf successfully!")
            return 1
        elif request.status_code == 400:
            print("\t\tERROR: The view ID in the URI doesn't correspond to a view available on the specified site.!!")
            return 0
        elif request.status_code == 401:
            print("\t\tERROR: Unauthorized!!")
            return 0
        elif request.status_code == 403:
            print("\t\tERROR: Forbidden to view pdf!!")
            return 0
        elif request.status_code == 404:
            print("\t\tERROR: Site, workbook or view not found!!")
            return 0
        elif request.status_code == 405:
            print("\t\tERROR: Invalid request method!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
    elif task == "download_workbook":
        if request.status_code == 200:
            print("\t\tDownloaded Workbook successfully!")
            return 1
        elif request.status_code == 403:
            print("\t\tERROR: Forbidden to view workbook!!")
            return 0
        elif request.status_code == 404:
            print("\t\tERROR: Site or workbook not found!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
    elif task == "query_view_data":
        if request.status_code == 200:
            print("\t\tQueried view data successfully!")
            return 1
        elif request.status_code == 401:
            print("\t\tERROR: Invalid token!!")
            return 0
        elif request.status_code == 403:
            print("\t\tERROR: Forbidden to view data!!")
            return 0
        elif request.status_code == 404:
            print("\t\tERROR: Site, workbook or view not found!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0

### Fn to sign in to Server with a password

In [4]:
def sign_in(username, password, site=""):
    body = {
        "credentials": {
            "name": username,
            "password": password,
            "site": {
                "contentUrl": site
            }
        }
    }
    response = requests.post(
        URL + '/auth/signin', 
        json=body, 
        verify=False, 
        headers={'Accept': 'application/json'}
    )
    
    status = check_error(response, "sign_in")
    if status:
        return response.json()['credentials']['site']['id'], response.json()['credentials']['token']
    else:
        return 0,0

### Fn to sign in to Server with a PAT

In [5]:
def sign_in_pat(username, password, site=""):
    body = {
        "credentials": {
            "personalAccessTokenName": username,
            "personalAccessTokenSecret": password,
            "site": {
                "contentUrl": site
            }
        }
    }
    response = requests.post(
        URL + '/auth/signin', 
        json=body, 
        verify=False, 
        headers={'Accept': 'application/json'}
    )
    
    status = check_error(response, "sign_in")
    if status:
        return response.json()['credentials']['site']['id'], response.json()['credentials']['token']
    else:
        return 0,0

### Fn to sign out from Server

In [6]:
def sign_out(site_id, token):
    response = requests.post(
        URL + '/auth/signout', 
        verify=False, 
        headers={'Accept': 'application/json',
                'X-Tableau-Auth': token}
    )
    status = check_error(response, "sign_out")
    return status

### Fn to find workbook id

In [7]:
def query_workbooks_site(site_id, token, workbook_name):
    response = requests.get(
        URL + '/sites/{}/workbooks?filter=name:eq:{}'.format(site_id, workbook_name), 
        verify=False, 
        headers={'Accept': 'application/json',
                'X-Tableau-Auth': token}
    )
    status = check_error(response, "query_workbooks_site")
    return response.json()

### Fn to generate list of views

In [8]:
def gen_list(orig_list):
    temp_list = orig_list.split(',')
    final_list = []
    for item in temp_list:
        item_name = (item.lstrip()).rstrip()
        final_list.append(item_name)
    return final_list

### Fn to find view id

In [9]:
def query_workbook(site_id, token, workbook_id, views_list):
    response = requests.get(
        URL + '/sites/{}/workbooks/{}'.format(site_id, workbook_id), 
        verify=False, 
        headers={'Accept': 'application/json',
                'X-Tableau-Auth': token}
    )
    view_ids = []
    for view in response.json()['workbook']['views']['view']:
        if view['name'] in views_list:
            view_ids.append(view['id'])
    status = check_error(response, "query_workbook")
    return view_ids, status

## Fn to query view as pdf

In [10]:
def query_view_pdf(site_id, token, view_id, filtername, filtervalue, filename, applyfilter):
    if applyfilter:
        response = requests.get(
            URL + '/sites/{}/views/{}/pdf?type=A4&orientation=Landscape&vf_{}={}'.format(site_id, view_id, urllib.parse.quote_plus(filtername), urllib.parse.quote_plus(filtervalue)), 
            stream=True, verify=False, 
            headers={'Accept': 'application/json',
                    'X-Tableau-Auth': token}
        )
    else:
        response = requests.get(
            URL + '/sites/{}/views/{}/pdf?type=A4&orientation=Landscape'.format(site_id, view_id), 
            stream=True, verify=False, 
            headers={'Accept': 'application/json',
                    'X-Tableau-Auth': token}
        )
    status = check_error(response, "query_view_pdf")
    with open(filename, 'wb') as f:
        for chunk in response:
            f.write(chunk)
    return status

### Fn to generate pdf with from temp pdf files

In [11]:
def gen_pdf_from_pdfs(filter_value_name, file_loc, pdf_filename, num_views):
    print("Developing PDF for {}...".format(filter_value_name))
    mergedObject = PdfFileMerger()

    for fileNumber in range(1,num_views):
        mergedObject.append(PdfFileReader(r'{}\temp_{}.pdf'.format(file_loc, fileNumber)))
        fileNumber += 1
    
    mergedObject.write(pdf_filename)
    
    for fileNumber in range(1,num_views):
        if os.path.exists(r'{}\temp_{}.pdf'.format(file_loc, fileNumber)):
            os.remove(r'{}\temp_{}.pdf'.format(file_loc, fileNumber))
            print("File deleted!")
        else:
            print("The file does not exist")
    
    print("Saved pdf")

### Fn to iterate over views and save pdfs

In [12]:
def iterate_views_pdf_unfiltered(site_id, token, view_ids, file_loc, pdf_filename):
    run = True
    attempt = 0
    while (run == True):
        count = 1
        for view in view_ids:
            apply_filter = False
            view_image_code = query_view_pdf(site_id, token, view, '', '',r"{}\temp_{}.pdf".format(file_loc, count), False)
            count+=1
            if (view_image_code == 1):
                run = False
                attempt+=1
            else:
                if attempt > 3:
                    run = False
                else:
                    run = True
                    attempt+=1
                    break
        
    gen_pdf_from_pdfs('All', file_loc, pdf_filename, count)

### Fn to iterate over views apply filters and save pdfs

In [13]:
def iterate_views_pdf_filtered(site_id, token, view_ids, apply_filter, filter_values, file_loc, pdf_filename):
    count = 1
    for view in view_ids:
        for filter_name in apply_filter:
            for filter_value in filter_values:
                view_pdf = query_view_pdf(site_id, token, view, filter_name, filter_value,r"{}\temp_{}.pdf".format(file_loc, count), True)
                count += 1


    gen_pdf_from_pdfs(filter_name, file_loc, pdf_filename, count)

## Fn Running of PDF Generation

In [14]:
def client_PDF_generation():
    Category = input('Which Company set of views do you want to export? Currys/Ikea/Staples - ')

    try:
        if Category == "":
            apply_filter = ""
            filter_values = ""
        else:
            apply_filter = gen_list(config[Category]['filtername'])
            filter_values = gen_list(config[Category]['applyfilters'])
    except KeyError:
        print("The following Company doesn\'t exist, or hasn\'t yet been configured - " + Category)
        print("Please try running the function again.")
        return
    except:
        print('Another error has occured, please try again.')
        return 

    pdf_filename = config["workbook_details"]["workbookname"] + ' - ' + Category + ' - ' + create_date_filename() + '.pdf'

    if apply_filter == "":
        print("Iterating over views with no filters...")
        iterate_views_pdf_unfiltered(site_id, token, query_workbook(site_id, token, workbook_response['workbooks']['workbook'][0]['id'], gen_list(config["workbook_details"]["viewnames"]))[0], config["workbook_details"]["download_loc"], pdf_filename)
    else:
        print("Iterating over views with filters applied...")
        iterate_views_pdf_filtered(site_id, token, query_workbook(site_id, token, workbook_response['workbooks']['workbook'][0]['id'], gen_list(config[Category]["viewnames"]))[0], apply_filter, filter_values, config["workbook_details"]["download_loc"], pdf_filename)
    return

### Fn to find year and week number

In [15]:
def create_date_filename():
    week = int(datetime.date.today().strftime("%V"))
    if week == 1:
        week = 52
        year = int(datetime.date.today().strftime("%Y"))-1
        date_part_filename = {}
    else:
        week-=1
        year = int(datetime.date.today().strftime("%Y"))
    return "{}{}".format(year, week)


## Variables

In [16]:
server = connection["server_connection"]["server"] # Enter site in format tableau.company.com without the https before it
site_content_url = connection["server_connection"]["site"] # This can be found from the URL of the content and if using the Default site then this will be blank
api_ver = connection["server_connection"]["api"] # This can be found from the Tableau Server REST API reference
URL = "https://{}/api/{}".format(server, api_ver)

## Checking login method and signing into Server

In [17]:
if connection["server_connection"]["loginmethod"] == 'PAT':
    username = connection["PAT_auth_details"]["token"]
    if username == "":
        print("No username mentioned in config file!! Please enter username in config file and try again")
        exit()
    password = connection["PAT_auth_details"]["token_secret"]
    if password == "":
        password = getpass("Enter your PAT for the Tableau Server: ")
    site_id, token = sign_in_pat(username, password, site_content_url)
    if token == 0:
        print("ZERO TOKEN A")
        exit()
elif connection["server_connection"]["loginmethod"] == 'Local':
    username = connection["local_auth_details"]["username"]
    if username == "":
        print("No username mentioned in config file!! Please enter username in config file and try again")
    password = connection["local_auth_details"]["password"]
    if password == "":
        password = getpass("Enter your password for the Tableau Server: ")
    site_id, token = sign_in(username, password, site_content_url)
    if token == 0:
        print("ZERO TOKEN B")
        exit()
else:
    print("Incorrect login method specified in config file under server_connection > loginmethod! Login method has to be Local or PAT")
    exit()

		User signed in successfully!


## Find workbook id from name

In [18]:
workbook_response = query_workbooks_site(site_id, token, urllib.parse.quote_plus(config["workbook_details"]["workbookname"]))

		Queried workbook name successfully!


## -- Generate PDFs --

## Iterate over views as PDFs and create PDF

In [21]:
client_PDF_generation()

Which Company set of views do you want to export? Currys/Ikea/Staples - Ikea
Iterating over views with filters applied...
		Queried view pdf successfully!
		Queried view pdf successfully!
		Queried view pdf successfully!
		Queried view pdf successfully!
Developing PDF for Category...
File deleted!
File deleted!
File deleted!
File deleted!
Saved pdf


## Close files and sign out of server

In [ ]:
log_file.close()
sign_out(site_id, token)